# Anonymizing Private Data

The original Tweedsmuir Park Lodge (TPL) activity schedule contains personal guest and staff information that is not included in this repository. To showcase the automation tools within the `Activity Schedule`, this notebook implements an anonymisation pipeline, mapping original names and reservation IDs to randomly generated replacements. 

Anonymous names are generated from a [Social Security Administration USA](https://www.ssa.gov/OACT/babynames/limits.html) dataset, which contains all baby names in the USA since 1880.


## Installs & Environments

In [ ]:
#-- Packages

# Operational
import numpy as np
import pandas as pd
from pathlib import Path
import sys
import random
import re

#-- Directories
nb_dir = Path.cwd()
REPO_ROOT = nb_dir.parent
data_dir = REPO_ROOT / 'Data/'

if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))


## Load Data

In [ ]:
# TPL Activity Schedule File

# File path 
activity_sch_path = data_dir/'TPL Summer Activity Schedule - Schedule - Reservations.csv'

act_sch = pd.read_csv(activity_sch_path)

act_sch.shape

## Create Random Name List


In [ ]:
# Data Source
USA_names = data_dir/'USA_names/'

# Get list of all txt files
files = sorted(f for f in USA_names.glob("yob*.txt")
               if int(f.stem[3:]) >= 1970      # f.stem is like 'yob1977' → [3:] = '1977'
)

cols = ["name", "gender", "count"]  # columns in each file

# Read and keep only name + gender from each file
df_list = []
for f in files:
    df = pd.read_csv(f, header=None, names=cols)
    df_list.append(df[["name", "gender"]])

# Combine into one big DataFrame
name_list = pd.concat(df_list, ignore_index=True)

name_list.head()
print(f'Name List Shape: {name_list.shape}')

In [ ]:
# Clean name list for use case


name_list_common = (                    # Common names for easy interpretation
    name_list
        .groupby(['name', 'gender'])
        .size()                         # count how many times each (name, gender) appears
        .reset_index(name='count')
        .query('count >= 50')           # Name has appeared >= 50 years in USA (Since 1970)
        .sort_values(['gender', 'name'])
        .reset_index(drop=True)
        [['name', 'gender']]            # keep just the columns you care about
)

name_list_common.head()
print(f"Common Name List Shape: {name_list_common.shape}")

## View Data

In [ ]:
print('All columns in Activity Schedule \n')
for col in act_sch.columns:
    print(col)

# Assign variable column names
reservations_col = 'Client Name'
reservations_subdata_col = 'Client Name.1'
date_col = 'date'
am_guides_col = 'AM Guide'
pm_guides_col = 'PM Guide'
reservation_notes_col = 'NOTES/REQUESTS'

## Anonymize Reservations

In [ ]:
# Mask: rows that look like "4xxxxxxx ..." (8 digits starting with 4)
mask = act_sch[reservations_col].str.match(r'^4\d{7}\s*[-–]?\s*', na=False)

# Extract the 8-digit ID from those rows
orig_ids = act_sch.loc[mask, reservations_col].str.extract(r'^(4\d{7})')[0]

# Unique reservation IDs
unique_ids = orig_ids.unique()
n_ids = len(unique_ids)
print(f"Unique reservations found: {n_ids}")


In [ ]:
# Generate one random full name per unique reservation
from Scripts.anonymize_utils import random_name_generator
new_names = random_name_generator(name_list_common, n_ids,seed_number=10)

# Generate unique new 8-digit IDs starting with '4'
new_ids = random.sample(range(40000000, 50000000), n_ids)  # unique integers

# Build mapping: old_id -> "new_id – New Name"
mapping = {
    old_id: f"{new_id} - {name}"
    for old_id, new_id, name in zip(unique_ids, new_ids, new_names)
}


In [ ]:
# Reassign new names to guest reservations

def anonymize_cell(value):
    # Keep NaNs as-is
    if pd.isna(value):
        return value

    # Try to match an 8-digit ID starting with 4
    m = re.match(r'^(4\d{7})', str(value))
    if m:
        old_id = m.group(1)
        # If it's a known reservation, replace with our new "<id> - <name>"
        if old_id in mapping:
            return mapping[old_id]


    # Value is not null and does NOT match the pattern -> null it
    return np.nan


# Apply function
act_sch[reservations_col] = act_sch[reservations_col].apply(anonymize_cell)

# View mapped ID values
print(f"'Old ID', 'New ID' - 'Anonymous Name'")
list(mapping.items())[:5] # show a few old->new mappings


#### Reservation Sub Data

In [ ]:
# Only rows where Client Name.1 is not null
mask = act_sch[reservations_subdata_col].notna()

act_sch.loc[mask, reservations_subdata_col] = (
    act_sch.loc[mask, reservations_col].astype(str).str.strip()
    + ' '
    + act_sch.loc[mask, 'Date'].astype(str)
)

### Reservation Notes

In [ ]:
act_sch[reservation_notes_col] = np.nan # Make all NaN values

## Anonymize Guides

In [ ]:
guide_cols = [am_guides_col, pm_guides_col]

guide_names = (
    act_sch[guide_cols]
        .stack()                        # AM guides + PM guides into one long Series
        .dropna()                       # remove NaNs
        .astype("string").str.strip()   # clean whitespace
        .unique()                       # unique names
)


In [ ]:
n_guides = len(guide_names)

from Scripts.anonymize_utils import first_name_generator
fake_guide_names = first_name_generator(name_list_common, n_guides,seed_number=10)

In [ ]:
# Map original guide -> fake guide
guide_map = dict(zip(guide_names, fake_guide_names))
guide_map["No Guide"] = "No Guide"

# Apply to the column (NaNs stay NaN)

act_sch[am_guides_col] = (  # AM guides column
    act_sch[am_guides_col]
        .str.strip()        # keeps NaNs as NaN
        .map(guide_map)     # names in map -> fake name, others -> NaN
        .fillna('')         # anything not mapped (or NaN) becomes ''
)


act_sch[pm_guides_col] = (  # PM guides column
    act_sch[pm_guides_col]
        .str.strip()        # keeps NaNs as NaN
        .map(guide_map)     # names in map -> fake name, others -> NaN
        .fillna('')         # anything not mapped (or NaN) becomes ''
)

## Format sheet

Output is for Google Sheets

In [ ]:
# All Nan values into blanks
act_sch = act_sch.fillna('')

# All rows with no reservations become blank

# 1. Column index of reservations_col
col_idx = act_sch.columns.get_loc(reservations_col)

# 2. All columns to the right of reservations_col
cols_to_right = act_sch.columns[col_idx + 1:]

# 3. Rows where reservations_col is empty string
mask = act_sch[reservations_col] == ''

# 4. For those rows, blank out all columns to the right
act_sch.loc[mask, cols_to_right] = ''

### Export CSV

In [ ]:
# Anonymized Activity Schedule
act_sch.to_csv(data_dir/'Anonymized_Activity_Schedule.csv',index=False)

In [ ]:
# Anonymized guide list for Google Sheet referencing
df_guides = pd.DataFrame(fake_guide_names,columns=['Name'])

# Add 'No Guide' value
No_name_row = pd.DataFrame({'Name':['No Guide']})
df_guides = pd.concat([df_guides,No_name_row],ignore_index=True)

# Export guides list
df_guides.to_csv(data_dir/'Anonymized_Guides_Reference.csv')

